# Домашнее задание 9

## Моделирование работы системы массового обслуживания
### Цель и результат

Научиться создавать массивы псевдослучайных чисел с нужным распределением и познакомиться с основами компьютерного моделирования работы систем массового обслуживания.

### Задание

Есть парикмахерская, с 𝑛 парикмахерами и очередью. Предполагается, что поток клиентов,
желающих постричься – пуассоновский с интенсивностью 𝜆, поток постриженных 𝑖-м
парикмахером клиентов – пуассоновский с интенсивностью 𝜇𝑖

1. Выбрав значения 𝑛, 𝜆 и 𝜇𝑖, написать программу, моделирующую работу этой парикмахерской в течение рабочего дня с 10:00 до 20:00. Результатом работы программы может быть текстовый файл, каждая строка которого отражает событие в работе парикмахерской: появление нового клиента или окончание стрижки клиента. Файл может выглядеть примерно так (нули и единицы перед слешем показывают, какой из парикмахеров занят, а какой свободен; число после слеша показывает число клиентов в очереди):

  10:03 - 0 0 1 / 0
  
  10:11 - 1 0 1 / 0
  
  10:18 - 1 1 1 / 0
  
  10:24 - 1 1 0 / 0
  
  10:27 - 1 1 1 / 0
  
  10:34 - 1 1 1 / 1
  
  10:39 - 1 1 1 / 2
  
  10:42 - 1 1 1 / 1


2. Подобрать значения 𝑛, 𝜆 и 𝜇𝑖 так, чтобы результаты работы парикмахерской выглядели правдоподобно, т. е. чтобы не образовывалась гигантская очередь, стрижка не занимала бы 1-2 минуты и т.п.

3. Написать словами свои соображения о том, как должны соотноситься друг с другом интенсивность потока клиентов и суммарная интенсивность работы парикмахеров, чтобы парикмахерская работала оптимальным образом (с одной стороны без многолюдных очередей, с другой – без постоянного простоя парикмахеров).

In [1]:
import numpy as np
from scipy import stats
from enum import Enum
from datetime import datetime, timedelta
from queue import Queue
from tqdm.notebook  import tqdm

In [2]:
class NodeState(Enum):
    """Статусы узлов обслуживания"""
    # Свободен
    FREE = 0
    # Занят
    BUSY = 1

In [3]:
class PoissonServiceNode:
    """ Узел обслуживания Пуассоновского процесса
    
    Параметры
    ---------
    m : float
      Интенсивность обслуживания
    t : int
      Интервал времени интенсивности обслуживания
    exp_scale_procent : float
      Стандартное отклонение для генерации случайного значения времени занятости,
      которое расчитывается по формуле: exp_scale_procent * (t / m)
    """
    def __init__(self, m=1.0, t=60, exp_scale_procent=0.1):
        
        assert 0 <= exp_scale_procent <= 1
        
        self.m = m
        self.t = t
        
        self.exp_scale_procent = exp_scale_procent
        self.__state = NodeState.FREE
        
        # Если нода занята, то в данной переменной храним время, через которое
        # узел будет свободен после момента взятия заявки
        self.__time2release = 0
        
        # Количество обслуженных заявок
        self.__clients_count = 0
        
        # Общее время работы узла
        self.__total_worktime = 0
        
        # Расчитывает среднее время на обслуживание одного клиента
        loc_busy = self.t / self.m 
        # Список времени работы узла при занятии клиентов
        self.__node_busy_times = list(stats.expon.rvs(loc=loc_busy, scale=loc_busy*self.exp_scale_procent, size=100000))
        
    @property
    def time2release(self):
        """ Время, через которое узел будет освобожден
        после принятия заявки.
        """
        return self.__time2release
    
    @property
    def node_busy_times(self):
        """Список времени работы узла при занятии клиентов"""
        return self.__node_busy_times
    
    @property
    def state(self):
        """Состояние узла"""
        return self.__state
    
    @property
    def clients_count(self):
        """Количество обслуженных клиентов"""
        return self.__clients_count
    
    @property
    def total_worktime(self):
        """Общее время работы"""
        return self.__total_worktime
        
    def take(self):
        """Занять узел обслуживания"""
        
        assert self.__state == NodeState.FREE, "Узел уже занят"
        
        # Меняем статус узла на занятый
        self.__state = NodeState.BUSY
        
        # При каждом взятии заявки в работу поулчаем случайное число времени обслуживания
        self.__time2release = self.__node_busy_times.pop()
        
        # Увеличиваем счетчик обслуженных клиентов
        self.__clients_count += 1
        
    def release(self):
        """Освободить узел обслуживания"""
        # Меняем статус узла на свободный
        self.__state = NodeState.FREE
        
        # Учитываем время работы
        self.__total_worktime += self.__time2release
        
        # Сбрасываем время до окончания обслуживания клиента
        self.__time2release = 0
        
    def time_step(self, time):
        """ Временной шаг для узла обслуживания.
        Используется, если узел занят.
        Из свойства time2release вычитается значение time.
        Если time2release <= 0, то происходит операция освобождения узла 
        
        Параметры
        ---------
        time : float
          Время
        """
        if self.__state == NodeState.FREE:
            return
        
        if time >= self.__time2release:
            self.__total_worktime += self.__time2release
            self.__time2release = 0
        else:
            self.__time2release -= time
            self.__total_worktime += time
        
        if self.__time2release < 1e-12:
            self.release()

In [4]:
class BarbershopPoissonProcess:
    """ Класс для эмуляции Пуассоновского процесса в парикмахерской
    
    Параметры
    ---------
    lambda_ : float
      Интенсивность входящего потока клиентов.
    m : float или [float]
      Интенсивность обслуживания отдельного парикмахера. Если float,
      то будет одинаковым для всех узлов обслуживания.
      Если список, то для каждого узла индивидуальное значение.
      В случае списка его длина должна быть равна node_count
    t_client : int
      Промежуток времени для интенсивности потока клиентах, указывается в минутах.
    t_barbershop : int
      Промежуток времени для интенсивности обслуживания клиентов каждого парикмахера, указывается в минутах.
    node_count : int
      Количество узлов обслуживания.
    client_exp_scale_procent : float
      Стандартное отклонение для генерации случайного значения времени ожидания следующего клиента.
      По умолчанию 0.3
    barbershop_exp_scale_procent : float
      Стандартное отклонение для генерации случайного значения времени обслуживания клиента.
      По умолчанию 0.3
    start_hour : int
      Час начала работы парикмахерской
    end_hour : int
      Час окончания работы парикмахерской
    no_print : bool
      Запрещаем печать инфомрации в консоль при симуляции
    """
    
    def __init__(self, lambda_, m, t_client, t_barbershop, node_count=3, client_exp_scale_procent=0.3, 
                 barbershop_exp_scale_procent=0.3, start_hour=9, end_hour=18, no_print=False):
        
        assert 0 <= client_exp_scale_procent <= 1
        assert 0 <= barbershop_exp_scale_procent <= 1
        assert 0 <= start_hour <= 23
        assert 0 <= end_hour <= 23
        assert start_hour < end_hour
        
        self.lambda_ = lambda_
        self.t_client = t_client
        self.client_exp_scale_procent = client_exp_scale_procent
        self.no_print = no_print        
        
        if hasattr(m, '__iter__'):
            assert len(m) == node_count, "Кол-во значений интенсивности обслуживания должно быть равно кол-ву узлов"
            self.__barbers = [PoissonServiceNode(m=m_barber, t=t_barbershop, exp_scale_procent=barbershop_exp_scale_procent)
                              for m_barber in m]
        else:
            self.__barbers = [PoissonServiceNode(m=m, t=t_barbershop, exp_scale_procent=barbershop_exp_scale_procent)
                              for idx in range(node_count)]
        
        # Время начала работы заведения
        self.__start_worktime = datetime.strptime(f"{start_hour}:00", "%H:%M")
        # Время окончания работы заведения
        self.__end_worktime = datetime.strptime(f"{end_hour}:00", "%H:%M")
        # Текущее время работы заведения, используется для эмуляции работы
        self.__barbershop_current_time = self.__start_worktime
        
        # Очередю клиентов
        self.__queue_client = Queue()
        
        # Максимальное время ожидания парикмахера клиентом
        self.__max_barber_wait_time = 0
        
    def __len__(self):
        return len(self.__barbers)
    
    @property
    def max_barber_wait_time(self):
        return self.__max_barber_wait_time 
    
    def barbers_relax_time(self, format_='H'):
        """Максимальное время ожидания парикмахера со стороны клиента"""
        
        format_ = str(format_).strip().upper()
        assert format_ in ('H', 'M')
        
        if format_ == 'H':
            devider = 60
        elif format_ == 'M':
            devider = 1
             
        total_worktime = (self.__end_worktime - self.__start_worktime).seconds / 60
        
        return np.array([total_worktime/devider - barber.total_worktime/devider for barber in self.__barbers])
        
    def print_barbers_relax_time(self, format_='H'):
        """ ПЕЧАТЬ
        Максимальное время ожидания парикмахера со стороны клиента"""
        
        format_ = str(format_).strip().upper()
        assert format_ in ('H', 'M')
        
        if format_ == 'H':
            time_type = 'ч.'
        elif format_ == 'M':
            time_type = 'мин.'
            
        all_barbers_relax_times = self.barbers_relax_time(format_=format_)
             
        for idx, barber_relax_time in enumerate(all_barbers_relax_times):
            print(f"Парикмахер {idx+1} отдыхал {barber_relax_time} {time_type}")
            
    def barbers_work_time(self, format_='H'):
        if format_ == 'H':
            devider = 60
        elif format_ == 'M':
            devider = 1
     
        return np.array([barber.total_worktime/devider for barber in self.__barbers])
    
    def print_barbers_work_time(self, format_='H'):
        """Максимальное время ожидания парикмахера со стороны клиента"""
        
        if format_ == 'H':
            time_type = 'ч.'
        elif format_ == 'M':
            time_type = 'мин.'
            
        all_barbers_times = self.barbers_work_time(format_=format_)
        
        for idx, barber_time in enumerate(all_barbers_times):
            print(f"Парикмахер {idx+1} работал {round(barber_time, 2)} {time_type}")
                               
    def _get_sorted_barbers_by_release_time(self):
        """ Сортировка парикмахеров в порядке наибыстрейшего освобождения от обслуживания текущего клиента
        В первых рядах будут свободные парикмахеры, а дальше те, кто быстрее всего освободится
        
        Результат
        ---------
        sorted_barbers_by_free_time : list
          Формат [(id парикмахера, количество времени до освобождения)]
          У свободных парикмахеров время 0.
        """
        
        sorted_barbers_by_release_time = [(idx, barber.time2release) for idx, barber in enumerate(self.__barbers)]
        sorted_barbers_by_release_time = sorted(sorted_barbers_by_release_time, key=lambda x: x[1])
        
        return sorted_barbers_by_release_time
    
    def _get_average_service_time(self):
        """Найти среднее время обслуживания клиентов парикмахерами"""
        
        barbers_total_work_time = sum([barber.total_worktime for barber in self.__barbers])
        barbers_total_clients = sum([barber.clients_count for barber in self.__barbers])
        
        if barbers_total_clients == 0:
            return 0
        
        return round(barbers_total_work_time / barbers_total_clients, 2)
    
    def _get_count_free_barbers(self):
        """Поиск количества свободных парикмахеров"""
        return len([barber for barber in self.__barbers if barber.state == NodeState.FREE])
    
    def _take_free_burbers(self, count):
        """ Занять свободных парикмахеров
        
        Параметры
        ---------
        count : int
          Какое кол-во свободных парикмахеров нужно занять.
          Не может быть больше общего количества свободных парикмахеров.
        """
        
        sorted_barbers_by_release_time = self._get_sorted_barbers_by_release_time()
        assert len(list(filter(lambda x: x[1] == 0, sorted_barbers_by_release_time))) >= count
        
        busied_burbers = []
        for idx in range(count):
            burber_index = sorted_barbers_by_release_time[idx][0]
            self.__barbers[burber_index].take()
            self._delete_client_queue(1)
            busied_burbers.append(str(burber_index+1))
        if len(busied_burbers) == 1:
            self._print_status(comment=f'[ОБСЛУЖИВАНИЕ] Парикмахер {", ".join(busied_burbers)} взял нового клиента') 
        elif len(busied_burbers) > 1:
            self._print_status(comment=f'[ОБСЛУЖИВАНИЕ] Парикмахеры {", ".join(busied_burbers)} взяли новых клиентов') 
            
    def _process_client_queue(self):
        """ Обработать очередь клиентов
        
        Распределяет клиентов из очереди по свободным парикмахерам
        
        Результат
        ---------
        queue_client_size : int
          Размер клиентской очереди
        """
        
        # Если время работы парикмахерской закончилось или парикмехер не успеет обслужить клиента 
        # до закрытия заведения, то очередь не будет обслужена/ И такая очередь полностью очищается.
        if self.__barbershop_current_time >= self.__end_worktime \
        or (self.__end_worktime - self.__barbershop_current_time).seconds/60  < self._get_average_service_time():
            self.__queue_client = Queue()
            return self.__queue_client.qsize()
        
        count_free_barbers = self._get_count_free_barbers()
        queue_size = self.__queue_client.qsize()
        
        # Находим сколько можно занять парикмахеров, нельзя занять больше, чем свободно на текущий момент
        how_many_takes_barbers = queue_size if queue_size <= count_free_barbers else count_free_barbers
        
        # Занимаем свободных парикмахеров согласно нашим нуждам и ограничениям и освобождаем очередь
        if how_many_takes_barbers > 0:
            self._take_free_burbers(how_many_takes_barbers)
        
        return self.__queue_client.qsize()
            
    def _add_client_queue(self, count):
        """ Добавление клиентов в очередь
        
        Параметры
        ---------
        count : int
          Кол-во пользователей для добавления
        """
        for _ in range(count):
            self.__queue_client.put(1)
        self._calc_current_max_barber_wait_time()
            
    def _delete_client_queue(self, count):
        """ Удаление клиентов из очереди
        
        Параметры
        ---------
        count : int
          Кол-во пользователей для удаления
        """
        for _ in range(count):
            self.__queue_client.get()  
            
    def _add_client_queue_and_process(self, count):
        """ Добавление клиентов в очередь
        
        Параметры
        ---------
        count : int
          Кол-во пользователей для добавления
        """
    
        queue_size_before = self.__queue_client.qsize()
        self._add_client_queue(count)
        queue_size_before_process = self.__queue_client.qsize()
        queue_size_after_process = self._process_client_queue()
            
        # Если у нас свободных парикмахеров меньше, чем требуется, то пишем лог
        if queue_size_after_process > queue_size_before:
            self._print_status(comment=f'[ОЧЕРЕДЬ] Очередь клиентов увеличилась на {queue_size_after_process - queue_size_before}')
            
        # Определяем были ли взяты хоть какие-то клиенты в работу
        clients_are_taken = queue_size_after_process < queue_size_before_process
            
        return self.__queue_client.qsize(), clients_are_taken
        
    def _do_barbershop_time_step(self, time):
        """ Делаем временной шал вперед для заведения и парикмахеров
        
        Уменьшаем время освобождения каждого парикмахера на указанное время.
        Если время занятности становится <= 0, то делаем парикмахера свободным.
        А также увеличиваем время работы парикмахерской на указанный шаг.
        """
        
        self.__barbershop_current_time += timedelta(minutes=time)
        
        for barber_idx, barber in enumerate(self.__barbers):
            barber_state_before = barber.state
            
            # Если время занятости barber.time2release в рамках шага станет <= 0, то парикмахер освободится
            barber.time_step(time) 
            barber_state_after = barber.state
            if barber_state_after != barber_state_before:
                self._print_status(comment=f'[ОБСЛУЖИВАНИЕ] Парикмахер {barber_idx+1} обслужил клиента')      
            
    def _print_status(self, comment=None):
        """ Печатаем текущий статус заведения
        """
        if self.no_print:
            return
        
        barbers_status_text = " ".join([str(int(barber.state == NodeState.BUSY)) for barber in self.__barbers])
        text = f"{datetime.strftime(self.__barbershop_current_time, '%H:%M')}: {barbers_status_text} / {self.__queue_client.qsize()}"
        text += f' # {comment}' if comment else ""
        print(text)
        
    def _get_barbershop_current_state(self):
        """ Текущее состояние заведения
        
        Результат
        ---------
        barbershop_current_state : tuple
          (Кол-во свободных парикмахеров, длина очереди клиентов, кол-во обслуженных клиентов)
        """
        
        count_free_burbers = self._get_count_free_barbers()
        client_queue_length = self.__queue_client.qsize()
        burbers_client_counts = sum([barber.clients_count for barber in self.__barbers])
        
        return count_free_burbers, client_queue_length, burbers_client_counts
    
    def __close_barbershop(self):
        """Дообслуживание клиентов после закрытия заведения
        Очередь очищается, парикмахеры заканчивают обслуживание клиентов
        
        """
        assert self.__barbershop_current_time >= self.__end_worktime
        # Очищаем очередь
        self.__queue_client = Queue()
        while True:
            sorted_barbers_by_release_time = self._get_sorted_barbers_by_release_time()
            if all([barber_release_time == 0 for _, barber_release_time in sorted_barbers_by_release_time]):
                break
            sorted_busy_barbers_by_release_time = list(filter(lambda x: x[1] != 0, sorted_barbers_by_release_time))
            first_barber_index, first_barber_release_time = sorted_busy_barbers_by_release_time[0]
            self._do_barbershop_time_step(time=self.__barbers[first_barber_index].time2release)
            
    def _pop(self, list_, size=1):
        for _ in range(size):
            _ = list_.pop()
            
    def _calc_current_max_barber_wait_time(self):
        """ Поиск максимального времени ожидания свободного парикмахера клиентом из очереди в текущий момент времени"""
        
        queue_size = self.__queue_client.qsize()
        free_burbers_count = self._get_count_free_barbers()
        total_burbers_count = len(self)
        
        # Если все парикмахеры могут обслужить очередь, то время ожидания равно 0
        if free_burbers_count >= queue_size:
            return
        
        if queue_size <= total_burbers_count:
            sorted_barbers_by_release_time = self._get_sorted_barbers_by_release_time()
            # В данном случае максимальное время ожидания - ожидание последнего клиента в очереди
            current_max_barber_wait_time = self.__barbers[sorted_barbers_by_release_time[queue_size-1][0]].time2release
        else:
            level_search = queue_size // total_burbers_count
            barbers_level_release_times = [(idx, barber.node_busy_times[-level_search]) 
                                           for idx, barber in enumerate(self.__barbers)]
            barbers_level_release_times = sorted(barbers_level_release_times, key = lambda x: x[1])[0]
            
            barber_idx = barbers_level_release_times[0]
            current_max_barber_wait_time = sum(self.__barbers[barber_idx].node_busy_times[-level_search:-1])
            
            
        if current_max_barber_wait_time > self.__max_barber_wait_time:
            self.__max_barber_wait_time = round(current_max_barber_wait_time, 2) 
            
    def _push_queue_for_time_limit(self, time_limit):
        """ Протолкнуть очередь клиентов к парикмахерам не более чем на указанное количество минут работы
        
        Параметры
        ---------
        time_limit : float
          Лимит времени на который можно увеличить рабочее время парикмахерской
          
        Результат
        ---------
        cum_time_step : float
          Количество минут, на которое шагнуло рабочее время вперед
        """
        
        cum_time_step = 0
        
        while True:
            
            # Найдем ближайшего свободного парикмахера
            sorted_barbers_by_release_time = self._get_sorted_barbers_by_release_time()
            # Найдем время освобождения ближайшего парикмахера
            next_barber_wait_time = sorted_barbers_by_release_time[0][1]
            
            cum_time_step += next_barber_wait_time
            
            # Если общий шаг времени превышает лимит, то заканчиваем работу
            if cum_time_step > time_limit:
                cum_time_step -= next_barber_wait_time
                return cum_time_step
            
            # Сделаем шаг рабочего времени для получения хотя бы одного свободного парикмахера
            self._do_barbershop_time_step(time=next_barber_wait_time)
            
            # Должен быть хотя бы один свободный парикмахер
            assert self._get_count_free_barbers() >= 1
            
            # У нас появился как минимум один свободный парикмахер, пробуем обслужить клиентов без добавления в очередь
            queue_size = self._process_client_queue()
            
            # Если очередь стала пустая, то завершаем цикл
            if queue_size == 0:
                return cum_time_step
           
    def start_simulation(self):
        """ Запуск симуляции работы парикмахерской
        """
        
        # Сбрасываем очередь и начальное время перед стартом
        self.__queue_client = Queue()
        self.__barbershop_current_time = self.__start_worktime
        
        # Генерируем значения времени ожидания каждого клиента
        # На определенных этапах нам нужно будет заглядывать в значения будущих клиентов,
        # поэтому генерируем список заранее перед циклом
        loc_client_wait = self.t_client / self.lambda_
        clients_wait_time = stats.expon.rvs(loc=loc_client_wait, 
                                                scale=loc_client_wait*self.client_exp_scale_procent,
                                                size=1000000)
        # Конвертируем в список
        clients_wait_time = list(clients_wait_time)
        
        # Выполняем обслуживание клиентов до тех пор, пока не закроется заведение
        # После закрытия очередь
        while self.__barbershop_current_time < self.__end_worktime:
            
            # Получаем время прихода текущего клиента
            current_client_wait_time = clients_wait_time.pop()
            current_queue_size = self.__queue_client.qsize()
            
            # Если очередь не равна 0, то перед приходом текущего клиента
            # нужно продвинуть очередь на время не более current_client_wait_time (врежя ожидания текущего клиента)
            if current_queue_size != 0:
                push_time_step = self._push_queue_for_time_limit(time_limit=current_client_wait_time)
            else:
                push_time_step = 0
                
            current_client_wait_time -= push_time_step
            
            # Так как в Пуассоновском поток ординарный, невозможна ситуация прихода нескольких клиентов одновременно,
            # то при получении времени прихода следующего клиента мы можем сделать шаг рабочего времени парикмахерской
            # и уменьшить время до освобождения парикмахеров с шагом времени текущего клиента
            self._do_barbershop_time_step(time=current_client_wait_time) 
            # Теперь время ожидания секущего клиента равна 0, фиксируем для формальности
            current_client_wait_time = 0
            
            # Добавляем текущего клиента в очередь и пробуем отдать его парикмахеру свободному парикмахеру
            queue_size, clients_are_taken = self._add_client_queue_and_process(count=1)
            # Если клиент взят в работу, то переходим к следующему клиенту
            if clients_are_taken:
                continue
            
            # Обслужить текущего клиента не удалось, текущий клиент сел на стульчик и хочет попасть
            # к первому освободившемуся парикмахеру.
            # Нам требуется сделать временной шаг до первого свободного парикмахера
            # и отдать ему одного клиента из очереди.
            # Но есть момент, к тому времени, как освободится первый парикмахер, могут прийти еще клиенты.
            
            # Отсортируем парикмахеров по времени освобождения (от наименьшего времени к большему)
            sorted_barbers_by_release_time = self._get_sorted_barbers_by_release_time()
            # Найдем время освобождения ближайшего парикмахера
            next_barber_wait_time = sorted_barbers_by_release_time[0][1]
            
            # Теперь нам нужно проверить придут ли еще клиенты за время next_barber_wait_time
            # Если за это время придут клиенты, то нам нужно перевести время заведения к моменту прихода
            # добавить клиента в очередь и попробовать обслужить (для логирования)
            
            # Время ожидания следующих клиентов с накоплением
            cum_next_client_wait_time = 0
            # Счетчик пришедших клиентов во время ожидания освобождения следующего парикмахера
            came_clients_counter = 0
            
            # Пробегаемся по будущим клиентам. Так как мы сейчас находимся в моменте времени,
            # когда текущий клиент только что пришел, то в clients_wait_time будут точные времена ожидания следующих клиентов
            # по цепочке.
            for idx in (range(1, len(clients_wait_time))):
                
                # Считаем сколько ждать следующего клиента с учетом предыдущего
                cum_next_client_wait_time += clients_wait_time[-idx]
                
                # Если следующий клиент приходит позже, чем освободится первый парикмахер, то останавливаем цикл
                if cum_next_client_wait_time > next_barber_wait_time:
                    # Вычитаем время клиента, так как нам нужно сделать шаг по времени до его прихода
                    cum_next_client_wait_time -= clients_wait_time[-idx]
                    break
                
                # Клиент пришел раньше, чем освободится первый парикмахер, делаем шаг по времени к приходу этого клиента
                self._do_barbershop_time_step(time=clients_wait_time[-idx])
                assert self._get_count_free_barbers() == 0, "Ни один парикмахер не может быть свободным, сейчас {self._get_count_free_barbers()}"
                
                # Добавляем его в очередь и пробуем обслужить.
                # Свободных парикмахеров не будет, клиенты не будут обслужены, 
                # но зато мы зафиксируем в логе увеличение очереди.
                _ = self._add_client_queue_and_process(count=1)
                
                # Увеличиваем счетчик таких клиентов
                came_clients_counter += 1
                
            # Если бы добавили в очередь будущих клиентов, то надо удалить их из clients_wait_time
            if came_clients_counter:
                clients_wait_time = clients_wait_time[:-came_clients_counter]
                
            assert cum_next_client_wait_time <= next_barber_wait_time
            
            # Делаем шаг по времени работы парикмахерской ровно до того времени, как освободится первый парикмахер
            self._do_barbershop_time_step(time=next_barber_wait_time-cum_next_client_wait_time)
            
            assert self._get_count_free_barbers() >= 1, f"Должен быть свободен хотя бы 1 парикмахер, сейчас {self._get_count_free_barbers()}"
            
            # У нас появился как минимум один свободный парикмахер, пробуем обслужить клиентов без добавления в очередь
            queue_size = self._process_client_queue()
           
        # Ко времени закрытия заведения некоторые парикмахеры могут еще работать, 
        # завершим работу парикмахеров после закрытия парикмахерской
        self.__close_barbershop()

## 1 Запуск симуляции

Особенности симуляции:
- если клиент приходит после окончания работы заведения, то он обслужен не будет и в очередь не встанет;
- если клиент пришел в рабочее время парикмахерской, но парикмахеры не успеют его обслужить до конца рабочего дня, то клиент не будет обслужен и не попадет в очередь;
- Нумерация парикмахеров в логе начинается с 1;
- Парикмахеры работают без перерыва на обед;
- При распределении очереди не учитывается какой парикмахер работал больше, клиента отдаем первому освободившемуся парикмахеру.

### Проверка корректности работы симуляции

Првоерим корректность работы эмуляции. Для этого создадим большую очередь для двух парикмахеров.

Попробуем запустить симудяцию работы парикмахерской со следющими параметрами:
- Предполагаем, что один парикмахер в среднем обслуживает 1 клиента за 40 минут (m=1, t_barbershop=40)
- Планируется появление в среднем 1 клиента за 5 минут (lambda_ = 1, t_client=5)
- Работает 2 парикмахера (node_count = 2)

In [5]:
config = {
    'lambda_': 1,
    'm': 1,
    't_client': 5,
    't_barbershop': 40,
    'node_count': 2,
    'client_exp_scale_procent': 0.3,
    'barbershop_exp_scale_procent': 0.3,
    'start_hour': 9,
    'end_hour': 18,
    'no_print': False,  
}

In [6]:
barbershop = BarbershopPoissonProcess(**config)
barbershop.start_simulation()

09:06: 1 0 / 0 # [ОБСЛУЖИВАНИЕ] Парикмахер 1 взял нового клиента
09:11: 1 1 / 0 # [ОБСЛУЖИВАНИЕ] Парикмахер 2 взял нового клиента
09:16: 1 1 / 1 # [ОЧЕРЕДЬ] Очередь клиентов увеличилась на 1
09:22: 1 1 / 2 # [ОЧЕРЕДЬ] Очередь клиентов увеличилась на 1
09:28: 1 1 / 3 # [ОЧЕРЕДЬ] Очередь клиентов увеличилась на 1
09:33: 1 1 / 4 # [ОЧЕРЕДЬ] Очередь клиентов увеличилась на 1
09:40: 1 1 / 5 # [ОЧЕРЕДЬ] Очередь клиентов увеличилась на 1
09:45: 1 1 / 6 # [ОЧЕРЕДЬ] Очередь клиентов увеличилась на 1
09:52: 1 1 / 7 # [ОЧЕРЕДЬ] Очередь клиентов увеличилась на 1
09:59: 0 1 / 7 # [ОБСЛУЖИВАНИЕ] Парикмахер 1 обслужил клиента
09:59: 1 1 / 6 # [ОБСЛУЖИВАНИЕ] Парикмахер 1 взял нового клиента
10:08: 1 1 / 7 # [ОЧЕРЕДЬ] Очередь клиентов увеличилась на 1
10:13: 1 0 / 7 # [ОБСЛУЖИВАНИЕ] Парикмахер 2 обслужил клиента
10:13: 1 1 / 6 # [ОБСЛУЖИВАНИЕ] Парикмахер 2 взял нового клиента
10:19: 1 1 / 7 # [ОЧЕРЕДЬ] Очередь клиентов увеличилась на 1
10:25: 1 1 / 8 # [ОЧЕРЕДЬ] Очередь клиентов увеличилась на 1
10:33:

Исходя из лога видно, что парикмахеры работают без перерыва. Как только освобождаются, сразу берут нового клиента. Парикмахеры могут быть свободны только утром, когда клиенты не пришли. И вечером, когда придется отказать всем клиентам в очереди из-за необходимости вовремя уйти с работы.

Проверим статистику отдыха парикмахеров, которая считается методами set и time_step класса PoissonServiceNode. Логирование и учет времени работают раздельно, если симуляция корректна, то результаты счетчика количества минут работы каждого экземпляра PoissonServiceNode должны совпадать с инфомрацией в логе. Для удобства мы будем считать количество отдыха каждого парикмахера. Найдем обзее количество рабочих минут и вычтем из них время работы каждого парикмахера.

In [7]:
barbershop.print_barbers_relax_time(format_='M')

Парикмахер 1 отдыхал 21.954101008292696 мин.
Парикмахер 2 отдыхал 43.0112729702156 мин.


Как видно из результата, данные сходятся. Эти минуты отдыха приходятся на начало рабочего дня и его конец. А дальше парикмахеры работают без перерыва.

Теперь проверим максимальное время ожидания клиента свободного парикмахера в такой очереди:

In [8]:
print("Максимальное ожидание свободного парикмахера:", barbershop.max_barber_wait_time, 'мин.')

Максимальное ожидание свободного парикмахера: 1266.57 мин.


## 2 Подбор оптимальных параметров

Поставим для парикмахерской следующие метрики качества:
- клиенты не должны ожидать парикмахера более 10 минут;
- парикмахеры должны сразу же брать клиента из очереди по факту своего освобождения (соблюдается всегда в самом принципе эмуляции);
- парикмахеры могут отднохнуть, если очередь пуста, и если обслужили своих текущих клиентов (соблюдается всегда в самом принципе эмуляции).

Пусть в парикмахерской работают 4 парикмахера (node_count = 4). И предполагаем, что каждый парикмахер в среднем обслуживает одного человека за 40 минут (m=1, t_barbershop=40). Данные параметры приближены к реальным.

С помощью экспериментов были подобраны следующие оптимальные параметры интенсивности потока: 3.35 человека за 40 минут (lambda_ = 3.35, t_client = 40)

In [9]:
config = {
    'lambda_': 3.35,
    'm': 1,
    't_client': 40,
    't_barbershop': 40,
    'node_count': 4,
    'client_exp_scale_procent': 0.3,
    'barbershop_exp_scale_procent': 0.3,
    'start_hour': 9,
    'end_hour': 18,
    'no_print': False,  
}

In [10]:
barbershop = BarbershopPoissonProcess(**config)
barbershop.start_simulation()

09:12: 1 0 0 0 / 0 # [ОБСЛУЖИВАНИЕ] Парикмахер 1 взял нового клиента
09:27: 1 1 0 0 / 0 # [ОБСЛУЖИВАНИЕ] Парикмахер 2 взял нового клиента
09:44: 1 1 1 0 / 0 # [ОБСЛУЖИВАНИЕ] Парикмахер 3 взял нового клиента
10:00: 0 1 1 0 / 0 # [ОБСЛУЖИВАНИЕ] Парикмахер 1 обслужил клиента
10:00: 1 1 1 0 / 0 # [ОБСЛУЖИВАНИЕ] Парикмахер 1 взял нового клиента
10:13: 1 1 1 1 / 0 # [ОБСЛУЖИВАНИЕ] Парикмахер 4 взял нового клиента
10:45: 0 1 1 1 / 0 # [ОБСЛУЖИВАНИЕ] Парикмахер 1 обслужил клиента
10:45: 0 0 1 1 / 0 # [ОБСЛУЖИВАНИЕ] Парикмахер 2 обслужил клиента
10:45: 0 0 0 1 / 0 # [ОБСЛУЖИВАНИЕ] Парикмахер 3 обслужил клиента
10:45: 1 0 0 1 / 0 # [ОБСЛУЖИВАНИЕ] Парикмахер 1 взял нового клиента
10:57: 1 1 0 1 / 0 # [ОБСЛУЖИВАНИЕ] Парикмахер 2 взял нового клиента
11:14: 1 1 0 0 / 0 # [ОБСЛУЖИВАНИЕ] Парикмахер 4 обслужил клиента
11:14: 1 1 1 0 / 0 # [ОБСЛУЖИВАНИЕ] Парикмахер 3 взял нового клиента
11:34: 0 1 1 0 / 0 # [ОБСЛУЖИВАНИЕ] Парикмахер 1 обслужил клиента
11:34: 1 1 1 0 / 0 # [ОБСЛУЖИВАНИЕ] Парикмахер 1 взя

#### Посчитаем среднее время ожидания клиентом свободного парикмахера и среднее время отдыха парикмахеров

In [11]:
num_experiments = 100
max_barber_wait_time_list = []
barbers_relax_time = np.zeros(shape=config['node_count'])

config['no_print'] = True
for _ in tqdm(range(num_experiments)):
    barbershop = BarbershopPoissonProcess(**config)
    barbershop.start_simulation()
    max_barber_wait_time_list.append(barbershop.max_barber_wait_time)
    barbers_relax_time += barbershop.barbers_relax_time(format_='M')
    
print("Среднее максимальное ожидание свободного парикмахера:", round(np.mean(max_barber_wait_time_list),2), 'мин.')

for idx, barber_relax_time in enumerate(barbers_relax_time):
    print(f"Среднее время отдыха парикмахера {idx+1}:", barbers_relax_time[idx] / num_experiments, 'мин.')

  0%|          | 0/100 [00:00<?, ?it/s]

Среднее максимальное ожидание свободного парикмахера: 10.24 мин.
Среднее время отдыха парикмахера 1: 90.34503676790915 мин.
Среднее время отдыха парикмахера 2: 120.8482890581213 мин.
Среднее время отдыха парикмахера 3: 152.58210420668166 мин.
Среднее время отдыха парикмахера 4: 225.51795021226567 мин.


In [12]:
print("Интенсивность потока:", config['lambda_'])
print("Суммарная интенсивность обслуживания:", config['node_count']*config['m'])

Интенсивность потока: 3.35
Суммарная интенсивность обслуживания: 4


## 3 Объяснение соотношения интенсивности потока и интенсивности обслуживания

Ради эесперимента будем увеличивать интенсивность потока каждый раз в 2 раза и подбирать интенсивность обслуживания каждого парикмахера с сохранением нашей метрики времени ожидания не более 10 минут. 

И посмотрим как будет меняться суммарная интенсивность ожидания.

In [13]:
config = {
    'lambda_': 6.70,
    'm': 1.8,
    't_client': 40,
    't_barbershop': 40,
    'node_count': 4,
    'client_exp_scale_procent': 0.3,
    'barbershop_exp_scale_procent': 0.3,
    'start_hour': 9,
    'end_hour': 18,
    'no_print': True,  
}

max_barber_wait_time_list = []
barbers_relax_time = np.zeros(shape=config['node_count'])

config['no_print'] = True
for _ in tqdm(range(num_experiments)):
    barbershop = BarbershopPoissonProcess(**config)
    barbershop.start_simulation()
    max_barber_wait_time_list.append(barbershop.max_barber_wait_time)
    barbers_relax_time += barbershop.barbers_relax_time(format_='M')
    
print("Среднее максимальное ожидание свободного парикмахера:", round(np.mean(max_barber_wait_time_list),2), 'мин.')

for idx, barber_relax_time in enumerate(barbers_relax_time):
    print(f"Среднее время отдыха парикмахера {idx+1}:", barbers_relax_time[idx] / num_experiments, 'мин.')

  0%|          | 0/100 [00:00<?, ?it/s]

Среднее максимальное ожидание свободного парикмахера: 10.21 мин.
Среднее время отдыха парикмахера 1: 69.11019540281423 мин.
Среднее время отдыха парикмахера 2: 85.57006374284012 мин.
Среднее время отдыха парикмахера 3: 104.90269641423488 мин.
Среднее время отдыха парикмахера 4: 142.26619199833382 мин.


In [14]:
print("Интенсивность потока:", config['lambda_'])
print("Суммарная интенсивность обслуживания:", config['node_count']*config['m'])

Интенсивность потока: 6.7
Суммарная интенсивность обслуживания: 7.2


In [15]:
config = {
    'lambda_': 13.4,
    'm': 3.05,
    't_client': 40,
    't_barbershop': 40,
    'node_count': 4,
    'client_exp_scale_procent': 0.3,
    'barbershop_exp_scale_procent': 0.3,
    'start_hour': 9,
    'end_hour': 18,
    'no_print': True,  
}

max_barber_wait_time_list = []
barbers_relax_time = np.zeros(shape=config['node_count'])

config['no_print'] = True
for _ in tqdm(range(num_experiments)):
    barbershop = BarbershopPoissonProcess(**config)
    barbershop.start_simulation()
    max_barber_wait_time_list.append(barbershop.max_barber_wait_time)
    barbers_relax_time += barbershop.barbers_relax_time(format_='M')
    
print("Среднее максимальное ожидание свободного парикмахера:", round(np.mean(max_barber_wait_time_list),2), 'мин.')

for idx, barber_relax_time in enumerate(barbers_relax_time):
    print(f"Среднее время отдыха парикмахера {idx+1}:", barbers_relax_time[idx] / num_experiments, 'мин.')

  0%|          | 0/100 [00:00<?, ?it/s]

Среднее максимальное ожидание свободного парикмахера: 10.67 мин.
Среднее время отдыха парикмахера 1: 37.410496627103555 мин.
Среднее время отдыха парикмахера 2: 45.31332803631072 мин.
Среднее время отдыха парикмахера 3: 54.69967359894053 мин.
Среднее время отдыха парикмахера 4: 64.46330242985597 мин.


In [16]:
print("Интенсивность потока:", config['lambda_'])
print("Суммарная интенсивность обслуживания:", config['node_count']*config['m'])

Интенсивность потока: 13.4
Суммарная интенсивность обслуживания: 12.2


In [17]:
config = {
    'lambda_': 26.8,
    'm': 5.7,
    't_client': 40,
    't_barbershop': 40,
    'node_count': 4,
    'client_exp_scale_procent': 0.3,
    'barbershop_exp_scale_procent': 0.3,
    'start_hour': 9,
    'end_hour': 18,
    'no_print': True,  
}

max_barber_wait_time_list = []
barbers_relax_time = np.zeros(shape=config['node_count'])

config['no_print'] = True
for _ in tqdm(range(num_experiments)):
    barbershop = BarbershopPoissonProcess(**config)
    barbershop.start_simulation()
    max_barber_wait_time_list.append(barbershop.max_barber_wait_time)
    barbers_relax_time += barbershop.barbers_relax_time(format_='M')
    
print("Среднее максимальное ожидание свободного парикмахера:", round(np.mean(max_barber_wait_time_list),2), 'мин.')

for idx, barber_relax_time in enumerate(barbers_relax_time):
    print(f"Среднее время отдыха парикмахера {idx+1}:", barbers_relax_time[idx] / num_experiments, 'мин.')

  0%|          | 0/100 [00:00<?, ?it/s]

Среднее максимальное ожидание свободного парикмахера: 9.86 мин.
Среднее время отдыха парикмахера 1: 21.070654284851102 мин.
Среднее время отдыха парикмахера 2: 25.605499484469455 мин.
Среднее время отдыха парикмахера 3: 29.308525842572436 мин.
Среднее время отдыха парикмахера 4: 34.96545983153082 мин.


In [18]:
print("Интенсивность потока:", config['lambda_'])
print("Суммарная интенсивность обслуживания:", config['node_count']*config['m'])

Интенсивность потока: 26.8
Суммарная интенсивность обслуживания: 22.8


Из экспериментов можно сделать вывод, что при увеличении интенсивности потока общая интенсивность обслуживания должна увеличиться примерно в такое же количество раз.